# 8. Méthode split-apply-combine

La méthode **groupby()** permet d'agréger les données selon les valeurs identiques d'une ou plusieurs colonnes.

Elle renvoie un objet de type *DataFrameGroupBy* qui peut être interprété comme un dictionnaire d'objets de type *DataFrame* dont :
- les clefs sont les modalités des valeurs de la colonne (ou des colonnes) utilisée(s) pour éclater les données
- et les valeurs des sous-DataFrames  du DataFrame initial.

**Méthodologie** :
- **split** : division des données en sous-ensembles
- **apply** : application d'une fonction sur chaque groupe
- **combine** : agrégation des résultats

Voir la documentation : http://pandas.pydata.org/pandas-docs/stable/groupby.html

In [1]:
# import des modules usuels
import pandas as pnd
import numpy as np
import matplotlib.pyplot as plt

# commande magique pour l'affichage des graphiques
%matplotlib inline

# options d'affichage
pnd.set_option("display.max_rows", 16)
plt.style.use('seaborn-darkgrid')

In [2]:
# chargement des données
geo = pnd.read_csv("correspondance-code-insee-code-postal.csv",
                   sep=';',
                   usecols=range(11),
                  index_col="Code INSEE")
geo.sort_index(inplace=True)
geo.head()

,Code Postal,Commune,Département,Région,Statut,Altitude Moyenne,Superficie,Population,geo_point_2d,geo_shape
Code INSEE,,,,,,,,,,
01001,01400,L'ABERGEMENT-CLEMENCIAT,AIN,RHONE-ALPES,Commune simple,242.0,1565.0,0.8,"46.1534255214, 4.92611354223","{""type"": ""Polygon"", ""coordinates"": [[[4.926273..."
01002,01640,L'ABERGEMENT-DE-VAREY,AIN,RHONE-ALPES,Commune simple,483.0,912.0,0.2,"46.0091878776, 5.42801696363","{""type"": ""Polygon"", ""coordinates"": [[[5.430089..."
01004,01500,AMBERIEU-EN-BUGEY,AIN,RHONE-ALPES,Chef-lieu canton,379.0,2448.0,13.4,"45.9608475114, 5.3729257777","{""type"": ""Polygon"", ""coordinates"": [[[5.386190..."
01005,01330,AMBERIEUX-EN-DOMBES,AIN,RHONE-ALPES,Commune simple,290.0,1605.0,1.6,"45.9961799872, 4.91227250796","{""type"": ""Polygon"", ""coordinates"": [[[4.895580..."
01006,01300,AMBLEON,AIN,RHONE-ALPES,Commune simple,589.0,602.0,0.1,"45.7494989044, 5.59432017366","{""type"": ""Polygon"", ""coordinates"": [[[5.614854..."


In [3]:
# on fabrique un objet de type DataFrameGroupBy
regions = geo.groupby("Région")
type(regions)

pandas.core.groupby.DataFrameGroupBy

Un objet *DataFrameGroupBy* peut être vu comme un dictionnaire dont :
- les clefs sont les valeurs de la colonne utilisée pour éclater les données
- les valeurs sont des sous-DataFrame du *DataFrame* initial (sans la colonne ayant servie à éclater les données

In [6]:
# accès au dictionnaire des groupes
type(regions.groups)

dict

In [7]:
# accès au nombre de groupes
regions.ngroups

27

In [9]:
geo["Région"].nunique()

27

La méthode **size()** permet de calculer l'effectif de chaque groupe.

In [10]:
# effectifs des groupes
regions.size()

Région
ALSACE                         904
AQUITAINE                     2296
AUVERGNE                      1310
BASSE-NORMANDIE               1812
BOURGOGNE                     2046
BRETAGNE                      1270
CENTRE                        1841
CHAMPAGNE-ARDENNE             1954
                              ... 
MIDI-PYRENEES                 3020
NORD-PAS-DE-CALAIS            1545
PAYS DE LA LOIRE              1502
PICARDIE                      2291
POITOU-CHARENTES              1462
PROVENCE-ALPES-COTE D'AZUR     978
REUNION                         24
RHONE-ALPES                   2887
Length: 27, dtype: int64

On vérifie que la somme des effectifs vaut le nombre total de communes.

In [11]:
# somme des effectifs
regions.size().sum() == len(geo)

True

La méthode **describe()** fournit les informations statistiques sur les sous-groupes.

In [12]:
geo.describe()

,Altitude Moyenne,Superficie,Population
count,36742.000000,3.674200e+04,36742.000000
mean,277.813810,1.735786e+03,1.755484
std,290.435871,1.444783e+04,8.109130
min,0.000000,2.000000e+00,0.000000
25%,103.000000,6.450000e+02,0.200000
50%,185.000000,1.081000e+03,0.400000
75%,334.000000,1.850000e+03,1.100000
max,2713.000000,1.871833e+06,440.200000


In [13]:
# describe par groupe
regions.describe()

Altitude Moyenne                                 \
                                      count        mean         std    min   
Région                                                                       
ALSACE                                904.0  302.655973  164.141776  110.0   
AQUITAINE                            2296.0  138.384146  154.109311    2.0   
AUVERGNE                             1310.0  637.366412  282.575966  204.0   
BASSE-NORMANDIE                      1812.0  119.595475   76.193013    4.0   
BOURGOGNE                            2046.0  281.342131  102.654453   64.0   
BRETAGNE                             1270.0   88.930709   54.477878    1.0   
CENTRE                               1841.0  142.575231   50.122349   29.0   
CHAMPAGNE-ARDENNE                    1954.0  198.787615   90.452667   64.0   
...                                     ...         ...         ...    ...   
MIDI-PYRENEES                        3020.0  410.063907  334.601047   56.0   
NORD-PAS-DE-CALAIS                   1545.0   80.408414   51.027241    0.0   
PAYS DE LA LOIRE                     1502.0   79.102530   50.662278    2.0   
PICARDIE                             2291.0  110.384548   40.675101    4.0   
POITOU-CHARENTES                     1462.0   90.224350   55.216061    1.0   
PROVENCE-ALPES-COTE D'AZUR            978.0  691.507157  564.062330    1.0   
REUNION                                24.0  804.333333  378.722299   34.0   
RHONE-ALPES                          2887.0  589.151715  399.692666   53.0   

                                                          Population  \
                              25%    50%      75%     max      count   
Région                                                                 
ALSACE                      186.0  253.0   361.00   928.0      904.0   
AQUITAINE                    57.0  103.0   175.00  1738.0     2296.0   
AUVERGNE                    376.5  613.0   871.75  1354.0     1310.0   
BASSE-NORMANDIE              52.0  114.0   176.25   344.0     1812.0   
BOURGOGNE                   200.0  260.5   350.00   655.0     2046.0   
BRETAGNE                     51.0   79.0   113.00   268.0     1270.0   
CENTRE                      110.0  132.0   166.00   424.0     1841.0   
CHAMPAGNE-ARDENNE           129.0  174.0   251.75   471.0     1954.0   
...                           ...    ...      ...     ...        ...   
MIDI-PYRENEES               198.0  292.0   497.25  2131.0     3020.0   
NORD-PAS-DE-CALAIS           34.0   75.0   122.00   241.0     1545.0   
PAYS DE LA LOIRE             43.0   71.0   103.75   274.0     1502.0   
PICARDIE                     82.0  105.0   135.00   247.0     2291.0   
POITOU-CHARENTES             44.0   89.0   127.00   296.0     1462.0   
PROVENCE-ALPES-COTE D'AZUR  237.0  550.0  1035.50  2519.0      978.0   
REUNION                     596.0  823.0  1044.75  1386.0       24.0   
RHONE-ALPES                 294.0  481.0   750.50  2713.0     2887.0   

                                        ...                   Superficie  \
                                 mean   ...        75%    max      count   
Région                                  ...                                
ALSACE                       2.040155   ...      1.500  271.7      904.0   
AQUITAINE                    1.396995   ...      1.000  236.7     2296.0   
AUVERGNE                     1.026336   ...      0.800  138.6     1310.0   
BASSE-NORMANDIE              0.812472   ...      0.700  109.3     1812.0   
BOURGOGNE                    0.802688   ...      0.600  152.1     2046.0   
BRETAGNE                     2.499921   ...      2.400  206.6     1270.0   
CENTRE                       1.378816   ...      1.100  135.2     1841.0   
CHAMPAGNE-ARDENNE            0.686131   ...      0.400  180.8     1954.0   
...                               ...   ...        ...    ...        ...   
MIDI-PYRENEES                0.947848   ...      0.600  440.2     3020.0   
NORD-PAS-DE-CALAIS           2.611586

La méthode **get_group()** permet d'obtenir le sous-*DataFrame* correspondant au groupe.

In [16]:
# accès à un groupe
regions.get_group("CORSE").head(7)

,Altitude Moyenne,Code Postal,Commune,Département,Population,Statut,Superficie,geo_point_2d,geo_shape
Code INSEE,,,,,,,,,
2A001,173.0,20167,AFA,CORSE-DU-SUD,2.8,Commune simple,1198.0,"41.9844089346, 8.79828936731","{""type"": ""Polygon"", ""coordinates"": [[[8.784625..."
2A004,136.0,20000,AJACCIO,CORSE-DU-SUD,64.3,Préfecture de région,8314.0,"41.9347926638, 8.70132275974","{""type"": ""Polygon"", ""coordinates"": [[[8.636808..."
2A006,244.0,20167,ALATA,CORSE-DU-SUD,3.0,Commune simple,3065.0,"41.9735186682, 8.73133206502","{""type"": ""Polygon"", ""coordinates"": [[[8.784316..."
2A008,488.0,20128,ALBITRECCIA,CORSE-DU-SUD,1.5,Commune simple,4599.0,"41.860474641, 8.87771179182","{""type"": ""Polygon"", ""coordinates"": [[[8.860380..."
2A011,728.0,20112,ALTAGENE,CORSE-DU-SUD,0.1,Commune simple,475.0,"41.7091975922, 9.07705100575","{""type"": ""Polygon"", ""coordinates"": [[[9.081959..."
2A014,168.0,20151,AMBIEGNA,CORSE-DU-SUD,0.1,Commune simple,621.0,"42.0878005416, 8.77383533017","{""type"": ""Polygon"", ""coordinates"": [[[8.786968..."
2A017,250.0,20167,APPIETTO,CORSE-DU-SUD,1.4,Commune simple,3459.0,"42.0032023283, 8.73355643393","{""type"": ""Polygon"", ""coordinates"": [[[8.788208..."


**Exercice**

Obtenir le groupe régional avec le plus petit effectif.

In [19]:
geo.groupby("Région").size().argmin()

'MAYOTTE'

La méthode **aggregate()** ou bien **agg()** permet d'agréger toutes les valeurs par regroupement en passant une ou plusieurs fonctions de calcul.

In [24]:
# somme de toutes les valeurs numériques agrégées
regions.agg(np.sum)

,Altitude Moyenne,Superficie,Population
Région,,,
ALSACE,273601.0,831863.0,1844.3
AQUITAINE,317730.0,4178042.0,3207.5
AUVERGNE,834950.0,2612534.0,1344.5
BASSE-NORMANDIE,216707.0,1774867.0,1472.2
BOURGOGNE,575626.0,3169039.0,1642.3
BRETAGNE,112942.0,2743887.0,3174.9
CENTRE,262481.0,3946105.0,2538.4
CHAMPAGNE-ARDENNE,388431.0,2570190.0,1340.7
...,...,...,...


In [25]:
# moyenne et écart type de toutes les valeurs agrégées
regions.agg([np.mean, np.std])

Altitude Moyenne                Superficie  \
                                       mean         std          mean   
Région                                                                  
ALSACE                           302.655973  164.141776    920.202434   
AQUITAINE                        138.384146  154.109311   1819.704704   
AUVERGNE                         637.366412  282.575966   1994.300763   
BASSE-NORMANDIE                  119.595475   76.193013    979.507174   
BOURGOGNE                        281.342131  102.654453   1548.894917   
BRETAGNE                          88.930709   54.477878   2160.540945   
CENTRE                           142.575231   50.122349   2143.457360   
CHAMPAGNE-ARDENNE                198.787615   90.452667   1315.348004   
...                                     ...         ...           ...   
MIDI-PYRENEES                    410.063907  334.601047   1510.269868   
NORD-PAS-DE-CALAIS                80.408414   51.027241    807.431715   
PAYS DE LA LOIRE                  79.102530   50.662278   2151.756325   
PICARDIE                         110.384548   40.675101    851.858577   
POITOU-CHARENTES                  90.224350   55.216061   1772.449384   
PROVENCE-ALPES-COTE D'AZUR       691.507157  564.062330   3239.408998   
REUNION                          804.333333  378.722299  10428.958333   
RHONE-ALPES                      589.151715  399.692666   1546.607897   

                                        Population             
                                    std       mean        std  
Région                                                         
ALSACE                       957.163127   2.040155  10.317211  
AQUITAINE                   2123.650800   1.396995   6.306459  
AUVERGNE                    1247.572770   1.026336   4.436420  
BASSE-NORMANDIE              684.901173   0.812472   3.173097  
BOURGOGNE                   1075.885575   0.802688   4.042821  
BRETAGNE                    1609.143176   2.499921   8.060555  
CENTRE                      1473.699429   1.378816   5.227198  
CHAMPAGNE-ARDENNE            972.845389   0.686131   4.784049  
...                                 ...        ...        ...  
MIDI-PYRENEES               1570.157061   0.947848   8.397871  
NORD-PAS-DE-CALAIS           594.144469   2.611586   8.642022  
PAYS DE LA LOIRE            1460.433128   2.355326   9.782091  
PICARDIE                     560.139710   0.835661   3.764407  
POITOU-CHARENTES            1306.414360   1.204856   4.000184  
PROVENCE-ALPES-COTE D'AZUR  3777.810043   4.998773  16.592725  
REUNION                     6300.543948  34.020833  35.375845  
RHONE-ALPES                 1543.580590   2.136301   7.236184  

[27 rows x 6 columns]

La méthode **apply()** applique la fonction spécifiée à chacun des groupes d'un objet *DataFrameGroupBy* et combine l'ensemble en un nouveau *DataFrame*.

In [30]:
# rang dans une région de chaque commune par population
def fct_rang(region):
    region["rang"] = region["Population"].rank(ascending=False)
    region["rang"] = region["rang"].astype(int)
    return region

df = regions.apply(fct_rang)
df.sort_values(["rang", "Population"], ascending=[True, False]).head(50)

,Altitude Moyenne,Code Postal,Commune,Département,Population,Statut,Superficie,geo_point_2d,geo_shape,rang
Code INSEE,,,,,,,,,,
31555,148.0,31000/31100/31200/31300/31400/31500,TOULOUSE,HAUTE-GARONNE,440.2,Préfecture de région,11809.0,"43.5963814303, 1.43167293364","{""type"": ""Polygon"", ""coordinates"": [[[1.461403...",1
06088,115.0,06000/06100/06200/06300,NICE,ALPES-MARITIMES,340.7,Préfecture,7391.0,"43.7119992661, 7.23826889465","{""type"": ""Polygon"", ""coordinates"": [[[7.308745...",1
44109,18.0,44000/44100/44200/44300,NANTES,LOIRE-ATLANTIQUE,282.0,Préfecture de région,6577.0,"47.2316356767, -1.54831008605","{""type"": ""Polygon"", ""coordinates"": [[[-1.51819...",1
67482,139.0,67000/67100/67200,STRASBOURG,BAS-RHIN,271.7,Préfecture de région,7809.0,"48.5712679849, 7.76752679517","{""type"": ""Polygon"", ""coordinates"": [[[7.770759...",1
34172,45.0,34000/34070/34080/34090,MONTPELLIER,HERAULT,255.1,Préfecture de région,5711.0,"43.6134409138, 3.86851657896","{""type"": ""Polygon"", ""coordinates"": [[[3.860837...",1
33063,9.0,33000/33100/33200/33300/33800,BORDEAUX,GIRONDE,236.7,Préfecture de région,4970.0,"44.8572445351, -0.57369678116","{""type"": ""Polygon"", ""coordinates"": [[[-0.57387...",1
75115,40.0,75015,PARIS-15E-ARRONDISSEMENT,PARIS,236.5,Chef-lieu canton,846.0,"48.8401554186, 2.29355937244","{""type"": ""Polygon"", ""coordinates"": [[[2.301320...",1
59350,27.0,59000/59160/59260/59777/59800,LILLE,NORD,226.8,Préfecture de région,3499.0,"50.6317183168, 3.04783272312","{""type"": ""Polygon"", ""coordinates"": [[[3.054628...",1
...,...,...,...,...,...,...,...,...,...,...


**Exercice**

Etablir une fonction qui calcule le top 10 des villes les plus habitées d'une région.

Utiliser la fonction pour limiter le *DataFrame* geo aux villes qui sont top 10 dans chaque région.

In [36]:
a = list(np.random.random(10))
a.sort()
a

[0.11612551674293803,
 0.19817033783572879,
 0.35843831767999446,
 0.36707114413824227,
 0.37129063102364024,
 0.72294560107558326,
 0.75151490807421639,
 0.79317397760350439,
 0.79669625057361582,
 0.81717161232002866]

In [32]:
def top3(region):
    return region.sort_values("Population", ascending=False).head(3)

geo.groupby("Région").apply(top3)

Code Postal  \
Région                     Code INSEE                                  
ALSACE                     67482                   67000/67100/67200   
                           68224                         68100/68200   
                           68066                               68000   
AQUITAINE                  33063       33000/33100/33200/33300/33800   
                           64445                               64000   
                           33281                               33700   
AUVERGNE                   63113                         63000/63100   
                           03185                               03100   
...                                                              ...   
PROVENCE-ALPES-COTE D'AZUR 83137                   83000/83100/83200   
                           13001       13080/13090/13100/13290/13540   
REUNION                    97411                               97400   
                           97415                               97460   
                           97416                               97410   
RHONE-ALPES                42218                   42000/42100/42230   
                           38185                         38000/38100   
                           69266                               69100   

                                                Commune           Département  \
Région                     Code INSEE                                           
ALSACE                     67482             STRASBOURG              BAS-RHIN   
                           68224               MULHOUSE             HAUT-RHIN   
                           68066                 COLMAR             HAUT-RHIN   
AQUITAINE                  33063               BORDEAUX               GIRONDE   
                           64445                    PAU  PYRENEES-ATLANTIQUES   
                           33281               MERIGNAC               GIRONDE   
AUVERGNE                   63113       CLERMONT-FERRAND           PUY-DE-DOME   
                           03185              MONTLUCON                ALLIER   
...                                                 ...                   ...   
PROVENCE-ALPES-COTE D'AZUR 83137                 TOULON                   VAR   
                           13001        AIX-EN-PROVENCE      BOUCHES-DU-RHONE   
REUNION                    97411            SAINT-DENIS               REUNION   
                           97415             SAINT-PAUL               REUNION   
                           97416           SAINT-PIERRE               REUNION   
RHONE-ALPES                42218          SAINT-ETIENNE                 LOIRE   
                           38185               GRENOBLE                 ISERE   
                           69266           VILLEURBANNE                 RHONE   

                                                           Région  \
Région                     Code INSEE                               
ALSACE                     67482                           ALSACE   
                           68224                           ALSACE   
                           68066                           ALSACE   
AQUITAINE                  33063                        AQUITAINE   
                           64445                        AQUITAINE   
                           33281                        AQUITAINE   
AUVERGNE                   63113                         AUVERGNE   
                           03185                         AUVERGNE   
...                                                           ...   
PROVENCE-ALPES-COTE D'AZUR 83137       PROVENCE-ALPES-COTE D'AZUR   
                           13001       PROVENCE-ALPES-COTE D'AZUR   
REUNION                    97411                          REUNION   
                           97415                          REUNION   
                           97416                          REUNION   
RHONE-ALPES                42218            

Il est possible de grouper selon plusieurs colonnes.

In [37]:
# groupes multiples
regions = geo.groupby(["Région", "Département"])
regions.describe()

Altitude Moyenne                          \
                                            count        mean         std   
Région      Département                                                     
ALSACE      BAS-RHIN                        527.0  241.631879  121.802016   
            HAUT-RHIN                       377.0  387.960212  177.299695   
AQUITAINE   DORDOGNE                        557.0  158.527828   62.845546   
            GIRONDE                         542.0   46.981550   28.425920   
            LANDES                          331.0   70.404834   39.174348   
            LOT-ET-GARONNE                  319.0  102.761755   39.865808   
            PYRENEES-ATLANTIQUES            547.0  270.349177  251.901880   
AUVERGNE    ALLIER                          320.0  331.328125  105.587393   
...                                           ...         ...         ...   
RHONE-ALPES AIN                             419.0  408.451074  242.373371   
            ARDECHE                         339.0  543.607670  306.112361   
            DROME                           369.0  504.672087  314.626959   
            HAUTE-SAVOIE                    294.0  809.149660  366.423874   
            ISERE                           533.0  607.003752  429.547830   
            LOIRE                           327.0  554.990826  217.835243   
            RHONE                           301.0  385.106312  166.809618   
            SAVOIE                          305.0  984.954098  614.960224   

                                                                        \
                                    min    25%    50%      75%     max   
Région      Département                                                  
ALSACE      BAS-RHIN              110.0  164.0  204.0   279.00   914.0   
            HAUT-RHIN             176.0  253.0  350.0   454.00   928.0   
AQUITAINE   DORDOGNE                9.0  115.0  156.0   196.00   385.0   
            GIRONDE                 2.0   21.0   45.0    66.00   139.0   
            LANDES                  5.0   36.5   68.0    96.00   187.0   
            LOT-ET-GARONNE         16.0   75.5  101.0   126.50   221.0   
            PYRENEES-ATLANTIQUES   12.0  138.5  211.0   300.00  1738.0   
AUVERGNE    ALLIER                204.0  260.0  305.0   379.25   915.0   
...                                 ...    ...    ...      ...     ...   
RHONE-ALPES AIN                   171.0  224.5  286.0   546.00  1198.0   
            ARDECHE                60.0  286.5  476.0   728.50  1315.0   
            DROME                  53.0  234.0  424.0   750.00  1397.0   
            HAUTE-SAVOIE          335.0  532.0  699.0  1019.75  2445.0   
            ISERE                 141.0  309.0  452.0   749.00  2557.0   
            LOIRE                 245.0  380.0  500.0   662.50  1163.0   
            RHONE                 166.0  241.0  332.0   523.00   815.0   
            SAVOIE                245.0  463.0  849.0  1375.00  2713.0   

                                 Population             ...                 \
                                      count      mean   ...     75%    max   
Région      Département                                 ...                  
ALSACE      BAS-RHIN                  527.0  2.079127   ...     1.5  271.7   
            HAUT-RHIN                 377.0  1.985676   ...     1.6  111.2   
AQUITAINE   DORDOGNE                  557.0  0.739677   ...     0.7   29.3   
            GIRONDE                   542.0  2.647601   ...     1.8  236.7   
            LANDES                    331.0  1.147130   ...     1.0   30.4   
            LOT-ET-GARONNE            319.0  1.033856   ...     0.8   33.9   
            PYRENEES-ATLANTIQUES      547.0  1.190128   ...     0.7   82.8   
AUVERGNE    ALLIER                    320.0  1.072813   ...     0.8   39.0   
...                                     ...       ...   ...     ...    ...   
RHONE-ALPES AIN                       419.0  1.405251   ...     1.5   39.6 

Il est possible d'utiliser la fonction **agg()** ou **aggregate()** sur une ou plusieurs colonnes en passant un dictionnaire de fonctions. On obtient alors un *DataFrame* avec les résultats ventilés par groupe et par colonne / clés de fonction.

In [38]:
# synthèses différenciées sur plusieurs colonnes
regions.agg({"Superficie": np.sum, "Population": np.sum, "Altitude Moyenne": np.mean})

Altitude Moyenne  Population  Superficie
Région      Département                                                   
ALSACE      BAS-RHIN                    241.631879      1095.7    479304.0
            HAUT-RHIN                   387.960212       748.6    352559.0
AQUITAINE   DORDOGNE                    158.527828       412.0    920777.0
            GIRONDE                      46.981550      1435.0   1014701.0
            LANDES                       70.404834       379.7    935097.0
            LOT-ET-GARONNE              102.761755       329.8    538097.0
            PYRENEES-ATLANTIQUES        270.349177       651.0    769370.0
AUVERGNE    ALLIER                      331.328125       343.3    736395.0
...                                            ...         ...         ...
RHONE-ALPES AIN                         408.451074       588.8    577259.0
            ARDECHE                     543.607670       313.6    555909.0
            DROME                       504.672087       483.2    655184.0
            HAUTE-SAVOIE                809.149660       727.8    459572.0
            ISERE                       607.003752      1197.7    786549.0
            LOIRE                       554.990826       745.7    479471.0
            RHONE                       385.106312      1700.1    325135.0
            SAVOIE                      984.954098       410.6    625978.0

[101 rows x 3 columns]

Il est également possible d'effectuer des regroupements selon des catégories numériques obtenues avec la fonction *digitize()*.

In [39]:
thd = pnd.read_excel("FranceTHD_Open_Data_Observatoire_Juin2015.xlsx",
                    sheetname="Communes",
                    header=1,
                    index_col="Code INSEE",
                    names=["Département", "Commune",
                                "1 Mbit", "3 Mbit", "8 Mbit", "30 Mbit", "100 Mbit",
                                "DSL 1 Mbit", "DSL 3 Mbit", "DSL 8 Mbit", "DSL 30 Mbit", "DSL 100 Mbit",
                                "Câble 1 Mbit", "Câble 3 Mbit", "Câble 8 Mbit", "Câble 30 Mbit", "Câble 100 Mbit",
                                "Fibre 1 Mbit", "Fibre 3 Mbit", "Fibre 8 Mbit", "Fibre 30 Mbit", "Fibre 100 Mbit"])
thd.head()

,Département,Commune,1 Mbit,3 Mbit,8 Mbit,30 Mbit,100 Mbit,DSL 1 Mbit,DSL 3 Mbit,DSL 8 Mbit,...,Câble 1 Mbit,Câble 3 Mbit,Câble 8 Mbit,Câble 30 Mbit,Câble 100 Mbit,Fibre 1 Mbit,Fibre 3 Mbit,Fibre 8 Mbit,Fibre 30 Mbit,Fibre 100 Mbit
Code INSEE,,,,,,,,,,,,,,,,,,,,,
01001,AIN,L'Abergement-Clémenciat,1.000,0.448,0.052,0.000,0.000,1.000,0.448,0.052,...,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.000,0.000,0.000
01002,AIN,L'Abergement-de-Varey,0.676,0.594,0.571,0.571,0.571,0.124,0.024,0.000,...,0.0,0.0,0.0,0.0,0.0,0.571,0.571,0.571,0.571,0.571
01004,AIN,Ambérieu-en-Bugey,1.000,0.966,0.794,0.234,0.000,1.000,0.966,0.794,...,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.000,0.000,0.000
01005,AIN,Ambérieux-en-Dombes,1.000,0.990,0.942,0.667,0.004,1.000,0.985,0.937,...,0.0,0.0,0.0,0.0,0.0,0.004,0.004,0.004,0.004,0.004
01006,AIN,Ambléon,1.000,1.000,1.000,0.934,0.934,1.000,1.000,1.000,...,0.0,0.0,0.0,0.0,0.0,0.934,0.934,0.934,0.934,0.934


In [40]:
# partition de [0,1] en 5 segments
bins = np.linspace(0, 1, 5)
bins

array([ 0.  ,  0.25,  0.5 ,  0.75,  1.  ])

In [45]:
thd["1 Mbit"]

Code INSEE
01001    1.000
01002    0.676
01004    1.000
01005    1.000
01006    1.000
01007    0.973
01008    1.000
01009    1.000
         ...  
97612    0.996
97613    0.999
97614    1.000
97615    1.000
97616    0.991
97617    0.990
97701    1.000
97801    0.993
Name: 1 Mbit, Length: 36693, dtype: float64

In [44]:
# catégorisation de la colonne "1 Mbit" selon la partition donnée
cat = np.digitize(thd["1 Mbit"], bins)
cat

array([5, 3, 5, ..., 4, 5, 4], dtype=int64)

Par défaut, il s'agit d'une partition semi-ouverte à droite (*right=False*) :
- [0, 0.25[
- [0.25, 0.5[
- [0.5, 0.75[
- [0.75, 1.0[
- [1.0, 1.0]

In [53]:
geo.groupby("Région").size()

Région
ALSACE                         904
AQUITAINE                     2296
AUVERGNE                      1310
BASSE-NORMANDIE               1812
BOURGOGNE                     2046
BRETAGNE                      1270
CENTRE                        1841
CHAMPAGNE-ARDENNE             1954
                              ... 
MIDI-PYRENEES                 3020
NORD-PAS-DE-CALAIS            1545
PAYS DE LA LOIRE              1502
PICARDIE                      2291
POITOU-CHARENTES              1462
PROVENCE-ALPES-COTE D'AZUR     978
REUNION                         24
RHONE-ALPES                   2887
Length: 27, dtype: int64

In [46]:
# regroupement selon la partition et calcul de la moyenne des valeurs des groupes
groups = thd.groupby(cat)
groups.mean()

,1 Mbit,3 Mbit,8 Mbit,30 Mbit,100 Mbit,DSL 1 Mbit,DSL 3 Mbit,DSL 8 Mbit,DSL 30 Mbit,DSL 100 Mbit,Câble 1 Mbit,Câble 3 Mbit,Câble 8 Mbit,Câble 30 Mbit,Câble 100 Mbit,Fibre 1 Mbit,Fibre 3 Mbit,Fibre 8 Mbit,Fibre 30 Mbit,Fibre 100 Mbit
1,0.053020,0.006919,0.003573,0.000606,0.000332,0.052688,0.006587,0.003241,0.000274,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000332,0.000332,0.000332,0.000332,0.000332
2,0.384078,0.048213,0.020293,0.003350,0.000581,0.383710,0.047632,0.019713,0.002769,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000581,0.000581,0.000581,0.000581,0.000581
3,0.647012,0.093624,0.043031,0.005817,0.003669,0.643564,0.089120,0.038526,0.002148,0,0.003728,0.003728,0.003728,0.002892,0.002892,0.000777,0.000777,0.000777,0.000777,0.000777
4,0.957697,0.488700,0.344005,0.097368,0.014134,0.956363,0.480850,0.333629,0.082615,0,0.008626,0.008626,0.008626,0.008539,0.006573,0.007655,0.007655,0.007655,0.007655,0.007655
5,1.000000,0.711897,0.540910,0.169414,0.028516,0.999988,0.704713,0.526087,0.137616,0,0.027997,0.027997,0.027997,0.027997,0.019013,0.011172,0.011172,0.011172,0.011172,0.011172


Regroupement et calcul des quantiles pour chaque colonne numérique avec la méthode **quantile()**.

In [47]:
# le premier décile de chaque colonne numérique par région
geo.groupby("Région").quantile(0.1)

0.1,Altitude Moyenne,Population,Superficie
Région,,,
ALSACE,156.0,0.30,280.8
AQUITAINE,27.0,0.10,496.0
AUVERGNE,284.0,0.10,676.4
BASSE-NORMANDIE,23.1,0.10,365.1
BOURGOGNE,177.5,0.10,509.5
BRETAGNE,30.0,0.40,585.9
CENTRE,91.0,0.20,734.0
CHAMPAGNE-ARDENNE,104.0,0.10,459.0
...,...,...,...


In [48]:
# les premier et dernier déciles de chaque colonne numérique par région
geo.groupby("Région").quantile([0.1, 0.9])

Altitude Moyenne  Population  Superficie
Région                                                                  
ALSACE                     0.1             156.0        0.30       280.8
                           0.9             532.7        3.30      1845.5
AQUITAINE                  0.1              27.0        0.10       496.0
                           0.9             258.0        2.30      3486.0
AUVERGNE                   0.1             284.0        0.10       676.4
                           0.9            1031.0        1.81      3665.1
BASSE-NORMANDIE            0.1              23.1        0.10       365.1
                           0.9             226.0        1.49      1748.4
...                                          ...         ...         ...
POITOU-CHARENTES           0.1              18.0        0.20       628.0
                           0.9             164.0        2.20      3223.5
PROVENCE-ALPES-COTE D'AZUR 0.1              78.0        0.10       790.2
                           0.9            1481.5       10.23      6615.4
REUNION                    0.1             308.2        6.06      3519.2
                           0.9            1270.0       75.78     18017.4
RHONE-ALPES                0.1             219.0        0.10       478.6
                           0.9            1105.4        3.84      2822.4

[54 rows x 3 columns]

**Exercice dirigé optionnel : analyse en composante principale avec StatsModels**

On va effectuer une ACP sur la moyenne des données THD par région.

1) Effectuer une jointure entre les 2 datasets thd et geo sur l'index des Code INSEE.

2) Regrouper par région, calculer la moyenne des valeurs numériques et restreigner aux 20 premières colonnes.

3) Importer la fonction **pca()** du module *statsmodels.sandbox.tools* et chercher dans la doc comment utiliser la fonction avec les données et obtenir des facteurs sur 2 dimensions

4) Reprendre le code du paragraphe 4.1.5 (cartographie avec annotations) pour afficher le résultat de l'ACP